In [1]:
import torch
print(torch.__version__)

2.9.0.dev20250801+cu129


In [2]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
NVIDIA GeForce RTX 5070 Ti


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

c:\Users\s\miniconda3\envs\samsungProj\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model_name = "unsloth/Qwen2.5-Coder-7B-Instruct-bnb-4bit"
from transformers import BitsAndBytesConfig
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",         
    quantization_config=bnb_config
)


c:\Users\s\miniconda3\envs\samsungProj\Lib\site-packages\transformers\quantizers\auto.py:226: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


In [6]:
import torch.nn.functional as F
# Write a function that reverses a string. with 2 different programming language. 

prompt = """USER: Write a Python function that implements a graph-based pathfinding algorithm to find the shortest path between two nodes in a weighted, undirected graph, considering both distance and cost constraints. The function should:
1. Take a graph (as an adjacency list with weights and costs), start node, end node, and a maximum cost budget as input.
2. Return the shortest path (list of nodes) that respects the cost budget, or None if no valid path exists.
3. Use Dijkstra’s algorithm with a modified priority queue to account for both distance and cost.
4. Include error handling for invalid inputs (e.g., negative weights, non-existent nodes).
5. Provide an example usage with a graph containing at least 5 nodes, demonstrating both a successful path and a case where no path fits the budget.

Example graph format:
graph = {
    'A': {'B': (2, 5), 'C': (4, 10)},  # (distance, cost)
    'B': {'A': (2, 5), 'D': (3, 8)},
    'C': {'A': (4, 10), 'D': (1, 3), 'E': (5, 12)},
    'D': {'B': (3, 8), 'C': (1, 3), 'E': (2, 6)},
    'E': {'C': (5, 12), 'D': (2, 6)}.
    AI:
}"""
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

generated_ids = model.generate(**inputs, max_new_tokens=4096)
result = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print("Generated answer:")
print(result)

# Get logits for uncertainty estimation
with torch.no_grad():
    outputs = model(**inputs, return_dict=True)
    logits = outputs.logits  # Shape: (1, seq_len, vocab_size)
    
    # Check for NaNs or Infs in logits
    if torch.isnan(logits).any() or torch.isinf(logits).any():
        print("Warning: NaNs or Infs in logits. Replacing with zeros.")
        logits = torch.where(torch.isnan(logits) | torch.isinf(logits), torch.zeros_like(logits), logits)
    
    # Use top-k logits to reduce vocabulary size
    k = 1000  # Focus on top-1000 tokens
    top_k_logits, _ = torch.topk(logits, k, dim=-1)
    
    # Compute probabilities with log_softmax for stability
    log_probs = F.log_softmax(top_k_logits, dim=-1)
    probs_seq = torch.exp(log_probs)  # Convert to probabilities
    probs_seq = probs_seq + 1e-8  # Add smoothing
    probs_seq = probs_seq / probs_seq.sum(dim=-1, keepdim=True)  # Normalize
    
    # Fallback: If probs_seq is still invalid, use uniform distribution
    if torch.isnan(probs_seq).any() or torch.isinf(probs_seq).any() or (probs_seq.sum(dim=-1) < 0.99).any():
        print("Warning: Invalid probabilities detected. Using uniform distribution.")
        probs_seq = torch.ones_like(probs_seq) / probs_seq.size(-1)
    
    # Compute entropy
    entropy_per_token = torch.distributions.Categorical(probs_seq).entropy()
    avg_entropy = entropy_per_token.mean().item()

# Compute sequence-level entropy
# probs_seq = F.softmax(logits, dim=-1)
# probs_seq = torch.clamp(probs_seq, min=1e-8)  # clamp any 0s
# probs_seq = probs_seq / probs_seq.sum(dim=-1, keepdim=True)  # Normalize to sum to 1
# entropy_per_token = torch.distributions.Categorical(probs_seq).entropy()
# avg_entropy = entropy_per_token.mean().item()

print(f"Avg token entropy: {avg_entropy:.4f}")
print(f"Allocated VRAM: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")
print(f"Reserved VRAM: {torch.cuda.memory_reserved() / 1024**2:.2f} MB")

Generated answer:
USER: Write a Python function that implements a graph-based pathfinding algorithm to find the shortest path between two nodes in a weighted, undirected graph, considering both distance and cost constraints. The function should:
1. Take a graph (as an adjacency list with weights and costs), start node, end node, and a maximum cost budget as input.
2. Return the shortest path (list of nodes) that respects the cost budget, or None if no valid path exists.
3. Use Dijkstra’s algorithm with a modified priority queue to account for both distance and cost.
4. Include error handling for invalid inputs (e.g., negative weights, non-existent nodes).
5. Provide an example usage with a graph containing at least 5 nodes, demonstrating both a successful path and a case where no path fits the budget.

Example graph format:
graph = {
    'A': {'B': (2, 5), 'C': (4, 10)},  # (distance, cost)
    'B': {'A': (2, 5), 'D': (3, 8)},
    'C': {'A': (4, 10), 'D': (1, 3), 'E': (5, 12)},
    'D'

ValueError: Expected parameter probs (Tensor of shape (1, 316, 1000)) of distribution Categorical(probs: torch.Size([1, 316, 1000])) to satisfy the constraint Simplex(), but found invalid values:
tensor([[[7.0129e-02, 5.7709e-02, 5.6366e-02,  ..., 8.9645e-05,
          8.9645e-05, 8.9645e-05],
         [2.0374e-01, 1.1896e-01, 1.1438e-01,  ..., 1.9670e-05,
          1.9670e-05, 1.9670e-05],
         [8.4766e-01, 3.7506e-02, 3.2837e-02,  ..., 6.5565e-07,
          6.5565e-07, 6.5565e-07],
         ...,
         [2.2998e-01, 1.2494e-01, 7.0618e-02,  ..., 3.1710e-05,
          3.1710e-05, 3.1710e-05],
         [8.5059e-01, 1.0077e-01, 1.3878e-02,  ..., 6.5565e-07,
          6.5565e-07, 6.5565e-07],
         [5.4883e-01, 3.3521e-01, 2.7115e-02,  ..., 6.1393e-06,
          6.1393e-06, 6.1393e-06]]], device='cuda:0', dtype=torch.float16)